# Бустинг

Будем предсказывать зарплату data scientist-ов в зависимости  от ряда факторов с помощью градиентного бустинга.

В датасете есть следующие признаки:



* work_year: The number of years of work experience in the field of data science.

* experience_level: The level of experience, such as Junior, Senior, or Lead.

* employment_type: The type of employment, such as Full-time or Contract.

* job_title: The specific job title or role, such as Data Analyst or Data Scientist.

* salary: The salary amount for the given job.

* salary_currency: The currency in which the salary is denoted.

* salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

* employee_residence: The country or region where the employee resides.

* remote_ratio: The percentage of remote work offered in the job.

* company_location: The location of the company or organization.

* company_size: The company's size is categorized as Small, Medium, or Large.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("ds_salaries.csv")
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


## Подготовка

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# делим выборку на train, val, test (80%, 10%, 10%)
train, val = train_test_split(df, test_size=0.2, random_state=42)
test, val = train_test_split(val, test_size=0.5, random_state=42)

In [ ]:
# salary_in_usd в качестве таргета
train_y = train['salary_in_usd']
train_X = train.drop('salary_in_usd', axis=1)

val_y = val['salary_in_usd']
val_X = val.drop('salary_in_usd', axis=1)

test_y = test['salary_in_usd']
test_X = test.drop('salary_in_usd', axis=1)

In [ ]:
# удаление признака salary, из-за которого возможен лик в данных, т.к. нам как раз нужно предсказывать зарплату
train_X.drop('salary', axis=1, inplace=True)
val_X.drop('salary', axis=1, inplace=True)
test_X.drop('salary', axis=1, inplace=True)

## Линейная модель

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# OneHot-кодирование
categorical_features = train_X.columns[(train_X.dtypes == 'object').values]
encoder = OneHotEncoder(sparse=False, handle_unknown='infrequent_if_exist')

In [ ]:
X_train_encoded = pd.DataFrame(encoder.fit_transform(train_X[categorical_features]),
                                  columns=encoder.get_feature_names_out(categorical_features),
                                  index=train_X.index)
X_test_encoded = pd.DataFrame(encoder.transform(test_X[categorical_features]),
                                  columns=encoder.get_feature_names_out(categorical_features),
                                  index=test_X.index)
X_valid_encoded = pd.DataFrame(encoder.transform(val_X[categorical_features]),
                                  columns=encoder.get_feature_names_out(categorical_features),
                                  index=val_X.index)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# обучение модели линейной регрессии
lr = LinearRegression()
lr.fit(X_train_encoded, train_y)
pred_y_lr = lr.predict(X_test_encoded)

In [ ]:
print('MAPE: ', mean_absolute_percentage_error(test_y, pred_y_lr))
print('RMSE: ', mean_squared_error(test_y, pred_y_lr, squared=False))

MAPE:  150674690.92530096
RMSE:  55272704734584.72


Качество модели очень низкое, высокое среднеквадратическое отклонение прогнозируемых значений от фактических подтверждает это, как и огромная средняя разница между прогнозом и фактом

## XGboost

Начнем с библиотеки xgboost.

Обучим модель `XGBRegressor` на тех же данных, что линейную модель, подобрав оптимальные гиперпараметры (`max_depth, learning_rate, n_estimators, gamma`, etc.) по валидационной выборке.

In [ ]:
import time

In [ ]:
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV

parameters = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 500, 1000],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.4, 0.6, 0.8],
    'colsample_bytree': [0.4, 0.6, 0.8],
    'min_child_weight': [1, 3, 5]
}

xgb = XGBRegressor()
grid_search = GridSearchCV(estimator=xgb, param_grid=parameters, cv=3, n_jobs=-1)
grid_search.fit(X_valid_encoded, val_y)

In [ ]:
%%time
best_xgb = XGBRegressor(**grid_search.best_params_)
best_xgb.fit(X_train_encoded, train_y)

In [ ]:
%%time
pred_y_xgb = best_xgb.predict(X_test_encoded)

CPU times: user 39.9 ms, sys: 0 ns, total: 39.9 ms
Wall time: 37.3 ms


In [ ]:
print('MAPE: ', mean_absolute_percentage_error(test_y, pred_y_xgb))
print('RMSE: ', mean_squared_error(test_y, pred_y_xgb, squared=False))

MAPE:  0.4134873202401893
RMSE:  42435.727065011626


Обучение модели заняло 4.28 s, предсказание - 39.9 ms, что говорит о довольно низкой скорости обучения и средней скорости предсказания. Полученные значения MAPE и RMSE указывают на достаточно высокую точность модели.

## CatBoost

Теперь библиотека CatBoost.

Обучим модель `CatBoostRegressor`, подобрав оптимальные гиперпараметры (`depth, learning_rate, iterations`, etc.) по валидационной выборке.

In [ ]:
!pip3 install catboost

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
parameters = {
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'iterations': [100, 200, 300],
    'l2_leaf_reg': [1, 3, 5],
    'random_strength': [1, 2, 5]
}

cbr = CatBoostRegressor()
grid_search = GridSearchCV(estimator=cbr, param_grid=parameters, cv=3, n_jobs=-1)
grid_search.fit(X_valid_encoded, val_y)

In [ ]:
%%time
best_cbr = CatBoostRegressor(**grid_search.best_params_)
best_cbr.fit(X_train_encoded, train_y)

In [ ]:
%%time
pred_y_cbr = best_cbr.predict(X_test_encoded)

In [ ]:
print('MAPE: ', mean_absolute_percentage_error(test_y, pred_y_cbr))
print('RMSE: ', mean_squared_error(test_y, pred_y_cbr, squared=False))

MAPE:  0.35569628969667394
RMSE:  49806.73946627879


Обучение модели заняло 189 ms, предсказание - 34.4 ms, что говорит о средней скорости обучения и высокой скорости предсказания. Полученные значения MAPE и RMSE указывают на достаточно высокую точность модели.

Для применения catboost моделей не обязательно сначала кодировать категориальные признаки, модель может кодировать их сама. Обучим catboost с подбором оптимальных гиперпараметров снова, используя pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметра cat_features.

In [ ]:
from catboost import Pool
from sklearn.model_selection import RandomizedSearchCV

categorical_features = ['experience_level', 'employment_type', 'job_title', 'salary_currency', 'employee_residence', 'company_location', 'company_size']
pool_train = Pool(data=train_X, label=train_y, cat_features=categorical_features)
pool_val = Pool(data=val_X, label=val_y, cat_features=categorical_features)
pool_test = Pool(data=test_X, label=test_y, cat_features=categorical_features)

In [ ]:
parameters = {
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'iterations': [100, 200, 300],
    'l2_leaf_reg': [1, 3, 5],
    'random_strength': [1, 2, 5]
}

cbr_pool = CatBoostRegressor()
random_search = RandomizedSearchCV(estimator=cbr_pool, param_distributions=parameters, n_iter=50, cv=3, n_jobs=-1)
random_search.fit(val_X, val_y)

In [ ]:
%%time
best_cbr_pool = CatBoostRegressor(**random_search.best_params_)
best_cbr_pool.fit(pool_train)

In [ ]:
%%time
pred_y_cbr_pool = best_cbr_pool.predict(pool_test)

In [ ]:
print('MAPE: ', mean_absolute_percentage_error(test_y, pred_y_cbr_pool))
print('RMSE: ', mean_squared_error(test_y, pred_y_cbr_pool, squared=False))

MAPE:  0.35569628969667394
RMSE:  49806.73946627879


Обучение модели стало занимать 2.9 ms, предсказание - 142.6 ms, что говорит о замедлении скорости работы модели, качество осталось прежним.

## LihtGBM

И наконец библиотека LightGBM - используйте `LGBMRegressor`, снова подберите гиперпараметры, оцените качество и скорость.


In [ ]:
from lightgbm import LGBMRegressor


parameters = {
    'max_depth' : [4, 6, 8],
    'learning_rate' : [0.01, 0.05, 0.1],
    'n_estimators' : [100, 150, 200],
    'num_leaves': [20, 50, 100],
    'reg_alpha': [0.0, 0.1, 0.5]
}

lgbmr = LGBMRegressor()
grid_search = GridSearchCV(estimator=lgbmr, param_grid=parameters, cv=3, n_jobs=-1)
grid_search.fit(X_valid_encoded, val_y)

In [ ]:
%%time
best_lgbmr = LGBMRegressor(**grid_search.best_params_)
best_lgbmr.fit(X_train_encoded, train_y)

In [ ]:
%%time
pred_y_lgbmr = best_lgbmr.predict(X_test_encoded)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
CPU times: user 8.95 ms, sys: 0 ns, total: 8.95 ms
Wall time: 11 ms


In [ ]:
print('MAPE: ', mean_absolute_percentage_error(test_y, pred_y_lgbmr))
print('RMSE: ', mean_squared_error(test_y, pred_y_lgbmr, squared=False))

MAPE:  0.3600051162058447
RMSE:  41409.55306088991


Обучение модели заняло 398 ms, предсказание - 11 ms, что говорит о низкой скорости обучения и высокой скорости предсказания. Полученные значения MAPE и RMSE указывают на достаточно высокую точность модели.

## Сравнение и выводы

По качеству предсказания лучший результат показала модель LightGBM, худший результат - линейная модель (очень большое значение отклонения, хотя и обучается быстро). Что касается скорости обучения, то выделяются следующие модели: линейная обучается действительно быстро, а вот XGboost пришлось доооолго ждать :( По скорости предсказания лидирует модель LightGBM, а CatBoost - это вообще кошмар (особенно с pool)!

Можно сказать, что лучше всего для данной задачи выбрать LightGBM, т.к. скорости обучения и предсказания удовлетворительные, а точность предсказаний высокая.

Гиперпараметры для разных моделей отличаются, однако имеют такие общие параметры, как learning_rate (скорость обучения модели), max_depth (максимальная глубина деревьев), min_child_weight (минимальный вес для разделения узла).